In [38]:
from urllib.parse import quote_plus
from sqlalchemy_utils import create_database, database_exists
from sqlalchemy import create_engine
from sqlalchemy import MetaData
import pymysql 
from sqlalchemy.orm import Session
from sqlalchemy import Table,Column,Integer, BIGINT, String, BigInteger, Boolean, Float, Unicode
from sqlalchemy import ForeignKey, Text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.dialects.mysql import LONGTEXT
meta = MetaData()
# Database connection details
user = 'root'
password = 'hannibal1999' #your password here
host = 'localhost'
port = '3306'
db_name = 'iranketab'

# Create the database if it doesn't exist
# character set to utf8mb4 so that to_sql can work with persian text
url = f'mysql://{user}:{password}@{host}:{port}/{db_name}?charset=utf8mb4'
if not database_exists(url):
    create_database(url)

# Connect to the database
engine = create_engine(url , pool_pre_ping=True)
conn = engine.connect()

In [39]:
session = Session(bind=engine)

In [40]:
summary = Table(
    'summary' , meta,
    Column('url_id' , Integer , primary_key=True ),
    Column('context' , Text),
    mysql_charset='utf8mb4',
    extend_existing=True,    
) 
meta.create_all(engine)
session.commit()

In [41]:
book = Table(
    'book' , meta, 
    Column('id' , Integer , primary_key=True),
    Column('url_id' , Integer , ForeignKey('summary.url_id') ),
    Column('title' , String(1000)),
    Column('en_title' , String(1000)),
    Column('price' , BIGINT),
    Column('ghat' , String(100)),
    Column('discount' , Integer),
    Column('ISBN' , String(100)),
    Column('cover' , String(100)),
    Column('page_count' , Integer),
    Column('solar_publish_year' , Integer),
    Column('ad_publish_year' , Integer),
    Column('edition' , Integer),
    Column('fastest_delivery' , String(15)),
    Column('rate' , Float),
    Column('available' , Boolean),      
    mysql_charset='utf8mb4',
    extend_existing=True,
)
meta.create_all(engine)
session.commit()

In [42]:
publication = Table(
    'publication' , meta,
    Column('id' , Integer , primary_key=True),
    Column('name' , String(100)),
    extend_existing=True,
) 
meta.create_all(engine)
session.commit()

In [43]:
publication_connector = Table(
    'publication_connector' , meta, 
    Column('id' , Integer , primary_key=True , autoincrement=True),
    Column('book_id' , Integer ,  ForeignKey('book.id') ),
    Column('publication_id' , Integer ,  ForeignKey('publication.id') ),
    extend_existing=True,
)
meta.create_all(engine)
session.commit()

In [44]:
tag = Table(
    'tag' , meta,
    Column('id' , Integer , primary_key=True ),
    Column('name' , String(100)),
    extend_existing=True,  
) 
meta.create_all(engine)
session.commit()

In [45]:
tag_book = Table(
    'tag_book' , meta, 
    Column('id' , Integer , primary_key=True , autoincrement=True),
    Column('book_id' , Integer ,  ForeignKey('book.id') ),
    Column('tag_id' , Integer ,  ForeignKey('tag.id') ),
    extend_existing=True,
)
meta.create_all(engine)
session.commit()

In [46]:
person = Table(
    'person' , meta,
    Column('id' , Integer , primary_key=True ),
    Column('name' , String(100)),
    extend_existing=True,
) 
meta.create_all(engine)
session.commit()

In [47]:
author = Table(
    'author' , meta,
    Column('id' , Integer , primary_key=True , autoincrement=True ),
    Column('book_id' , Integer,  ForeignKey('book.id' ) ),
    Column('person_id' , Integer  ,  ForeignKey('person.id')),
    extend_existing=True,
)
meta.create_all(engine)
session.commit()

In [48]:
translator = Table(
    'translator' , meta,
    Column('id' , Integer , primary_key=True , autoincrement=True),
    Column('person_id' , Integer  , ForeignKey('person.id')),
    Column('book_id' , Integer , ForeignKey('book.id') ),
    extend_existing=True,
)
meta.create_all(engine)
session.commit()

## Part 2: cleaning the data , inserting into database
### summary table

In [49]:
# read the data from csv
import pandas as pd
df_summary = pd.read_csv ('book_summary.csv' , encoding='UTF-8')

In [50]:
# we drop the extra \n s in the text, fix column types and column names so they would be passed by to_sql()
df_summary['text'] = df_summary['text'].str.strip().replace('\n', '')
df_summary = df_summary.astype({'book_id': 'int64'})
df_summary.rename(columns={'book_id' : 'url_id'}, inplace=True)
df_summary.rename(columns={'text' : 'context'}, inplace=True)
df_summary = df_summary.astype({'context': 'string'})
# fill nan values, pass data to csv
df_summary['context'] = df_summary['context'].fillna('NaN')
df_summary['context'] = df_summary['context'].str.replace('<NA>' , "NaN")

In [51]:
# pass the data to database
df_summary.to_sql('summary' , con=engine ,  if_exists='append' , index=False)

99919

### book table

In [52]:
# read data from csv
import numpy as np
df_book = pd.DataFrame(pd.read_csv('book.csv' , encoding='UTF-8'))

C:\Users\Parinaz\AppData\Local\Temp\ipykernel_13732\1708306972.py:3: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_book = pd.DataFrame(pd.read_csv('book.csv' , encoding='UTF-8'))


In [53]:
# strip unwanted \n s from different columns, stripping the unwanted spaces, cleaning additional unwanted text
df_book['title'] = df_book['title'].str.strip('\n')
df_book['ghat'] = df_book['ghat'].str.strip()
df_book['ghat'] = df_book['ghat'].str.strip('\n')
df_book['price'] = df_book['price'].str.strip().replace('\n' , '')
df_book['ISBN'] = df_book['ISBN'].str.strip().replace('\n' , '')
df_book['cover'] = df_book['cover'].str.strip().replace('\n' , '')
df_book['cover'] = df_book['cover'].apply(lambda x: x.strip() if len(str(x)) > 5 else x)
df_book['price'] = df_book['price'].str.replace("," , "").fillna(0)
df_book['edition'] = pd.to_numeric(df_book['edition'], errors='coerce').fillna(0.0)
df_book['page_count'] = df_book['page_count'].str.strip().replace('\n' , '').fillna('0')
df_book['page_count'] = df_book['page_count'].apply(lambda x: '0' if not str(x).isdigit() else x)
#df_book['page_count'] = df_book['page_count'].apply(lambda x: '0' if len(str(x)) > 5 else x)
df_book['solar_publish_year'] = df_book['solar_publish_year'].str.strip().replace('\n' , '')
df_book['solar_publish_year'] = df_book['solar_publish_year'].apply(lambda x: str(x).zfill(4) if str(x).isdigit() and len(str(x)) == 4 else '0')
df_book['ad_publish_year'] = df_book['ad_publish_year'].str.strip().replace('\n' , '')
df_book['ad_publish_year'] = df_book['ad_publish_year'].apply(lambda x: str(x).zfill(4) if str(x).isdigit() and len(str(x)) == 4 else '0')
df_book['fastest_delivery'] = df_book['fastest_delivery'].str.strip('\n')

# Strip "کتاب" from titles
df_book['title'] = df_book['title'].str.replace('کتاب', '')
# Remove content within parentheses 
# سمفونی مردگان (جیبی)
# to: سمفونی مردگان
df_book['title'] = df_book['title'].str.replace(r"\(.*\)", '' , regex=True)
df_book['title'] = df_book['title'].str.lstrip()
df_book['title'] = df_book['title'].str.rstrip()


# Replace NaN values in 'id' column with corresponding 'url_id' values
booktemp = df_book.copy()
booktemp['id'] = np.where(booktemp['id'].isnull() & ~booktemp['url_id'].isin(booktemp['id']), booktemp['url_id'], booktemp['id'])
# Verify that there are no duplicate values in 'id' column
booktemp.drop_duplicates(subset='id', keep='first', inplace=True)
df_book = booktemp

# adjusting column datatypes , fill nan values
# booktemp = df_book[df_book['id'].isnull()]
df_book.dropna(subset=['id'], inplace=True)
df_book = df_book.astype({'id': 'int64'})
df_book['discount'].fillna(0, inplace=True)
df_book = df_book.astype({'discount': 'int64'})
df_book['edition'].fillna(0, inplace=True)
df_book = df_book.astype({'edition': 'int64'})
df_book['solar_publish_year'].fillna(0, inplace=True)
df_book = df_book.astype({'solar_publish_year': 'int64'})
df_book['solar_publish_year'].fillna(0, inplace=True)
df_book = df_book.astype({'ad_publish_year': 'int64'})
df_book = df_book.astype({'available': 'bool'})
df_book = df_book.astype({'title': 'str'})
df_book = df_book.astype({'en_title': 'string'})
df_book = df_book.astype({'price': 'int64'})
df_book = df_book.astype({'ghat': 'string'})
df_book = df_book.astype({'ISBN': 'string'})
df_book = df_book.astype({'cover': 'string'})
df_book = df_book.astype({'page_count': 'int64'})
df_book = df_book.astype({'fastest_delivery': 'string'})


#### fixing noise datas

In [54]:
# PAGE COUNT MANUAL NOISE FIX
df_book.loc[df_book['id'] == 88550, 'page_count'] = 758 #edited from amazon website
df_book.loc[df_book['id'] == 95310, 'page_count'] = 680 #edited from other page counts on the internet
df_book.loc[df_book['id'] == 87053, 'page_count'] = 298 #majale - edited from previous ones
df_book.loc[df_book['id'] == 51430, 'page_count'] = 1724 #edited considering the price
df_book.loc[df_book['id'] == 121685, 'page_count'] = 970 #edited from other websites

In [58]:
# passing data to database
df_book.to_sql('book' , con=engine , if_exists='append' , index=False  )

113736

### person , author and translator tables

In [59]:
# read data from csv
import pandas as pd
df_author = pd.read_csv('outher.csv' , encoding='UTF-8')
df_translator = pd.read_csv('translator.csv' , encoding='UTF-8')

In [61]:
# strip the unwanted \n s
df_author['name'] = df_author['name'].str.strip('\n')
df_translator['name'] = df_translator['name'].str.strip('\n')
df_translator['name'] = df_translator['name'].str.replace('\n' , '')

# reading name list from df s, removing duplicates and make df_person
namelist = pd.DataFrame(df_translator['name'])
name_list = pd.DataFrame(df_author['name'])
names = pd.concat([namelist, name_list], ignore_index=True)
names.drop_duplicates(subset=['name'] , inplace=True )
names = names.assign(id=range(1, len(names) + 1))
names.reset_index(inplace=True , drop=True)
df_person = names
df_person.dropna(subset=['name'], inplace=True)
#df_translator.dropna(subset=['name'], inplace=True)
#df_author.dropna(subset=['name'], inplace=True)

# id_book column name from csv changed to pass to database
df_translator.rename(columns={'id_book' : 'book_id'}, inplace=True)
df_author.rename(columns={'id_book' : 'book_id'}, inplace=True)
# create relation with merge
df_author = df_author.merge(df_person[['name', 'id']], on='name', how='left')
df_author = df_author.rename(columns={'id': 'person_id'})

df_translator = df_translator.merge(df_person[['name', 'id']], on='name', how='left')
df_translator = df_translator.rename(columns={'id': 'person_id'})
# clean data, make sure there's no duplicates
df_author.drop_duplicates(subset=['person_id'] , inplace=True )
df_author.drop('name' , axis=1 , inplace=True)
#df_author.drop_duplicates(subset=['book_id'] , inplace=True )
df_translator.drop('name' , axis=1, inplace=True)

In [64]:
# pass the data to database
df_person.to_sql('person' , con=engine ,  if_exists='append' , index=False)

54767

In [65]:
# pass the data to database
df_author.to_sql('author' , con=engine ,  if_exists='append' , index=False)
# pass the data to database
df_translator.to_sql('translator' , con=engine ,  if_exists='append' , index=False)

40388

### publish_connector and publication tables:

In [67]:
# read data from csv
df_pub_connector = pd.read_csv('publisher.csv' , encoding='UTF-8')

In [68]:
# rename column from csv to pass data to database, strip unwanted \n s
df_pub_connector['name'] = df_pub_connector['name'].str.strip('\n')
df_pub_connector = df_pub_connector.rename(columns={'id_book': 'book_id'})
# create publication df, make sure there's no duplicates
df_publication = pd.DataFrame(df_pub_connector['name'])
df_publication.drop_duplicates(subset=['name'] , inplace=True )
df_publication = df_publication.assign(id=range(1, len(df_publication) + 1))
df_publication.reset_index(inplace=True , drop=True)

In [69]:
# make publication connector with merge
df_pub_connector = df_pub_connector.merge(df_publication[['name' , 'id']] , on='name' , how='left')
df_pub_connector = df_pub_connector.rename(columns={'id' : 'publication_id'})
df_pub_connector.drop('name' , axis=1 , inplace=True)

In [70]:
#pass the data to database
df_publication.to_sql('publication' , con=engine ,  if_exists='append' , index=False)
df_pub_connector.to_sql('publication_connector' , con=engine ,  if_exists='append' , index=False)
session.commit()

### tag and tag_book tables:

In [71]:
# Import the required libraries
import pandas as pd

# Read the tag and award CSV files
df_tag = pd.read_csv('tag.csv', encoding='UTF-8')
df_tag.drop('tag_id', axis=1, inplace=True)

df_tag['tag'] = df_tag['tag'].apply(eval)
df_tag['tag'] = df_tag['tag'].apply(lambda x: [i.strip() for i in x])

df_tag= df_tag.explode('tag')
df_tag_book = df_tag

In [72]:
# tag table creation
tags = pd.DataFrame(df_tag['tag'])
# we just want unique names for tags
tags.drop_duplicates(subset=['tag'] , inplace=True)
# creating id column for tags -> tag.id / tag_id
tags = tags.assign(id=range(1, len(tags) + 1))
tags.reset_index(inplace=True , drop=True)

# drop nan s in both tables
df_tag_book.dropna(subset=['tag'], inplace=True)
tags.dropna(subset=['tag'], inplace=True)

# rename columns , no name column needed in tag_book
df_tag_book = df_tag_book.merge(tags[['tag' , 'id']] ,  on='tag', how='left')
tags.rename(columns={'tag' : 'name'}, inplace=True)
df_tag_book.rename(columns={'id' : 'tag_id'}, inplace=True)
df_tag_book.drop('tag' , axis=1 , inplace=True)

In [74]:
tags.to_sql('tag', con=engine, if_exists='append', index=False)
df_tag_book.to_sql('tag_book', con=engine, if_exists='append', index=False)
session.commit()

519226